## specifically: stride time (cadence and stride length should be redundant)

In [1]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import pandas as pd
import pingouin as pg
import seaborn as sns
from biopsykit.questionnaires.utils import compute_scores, wide_to_long
from biopsykit.utils.dataframe_handling import convert_nan
from fau_colors import cmaps, register_fausans_font

from stressgait_analysis.dataset import StressGaitDataset

%matplotlib widget
%load_ext autoreload
%autoreload 2
from stressgait_analysis.gait_helper_functions import compute_HS
import numpy as np
import warnings
warnings.simplefilter(action='ignore')

In [2]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties_light)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
#plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

[(0.37254901960784315, 0.48627450980392156, 0.6392156862745098),
 (0.6666666666666666, 0.7647058823529411, 0.8196078431372549),
 (1.0, 0.8235294117647058, 0.44313725490196076),
 (0.37254901960784315, 0.7725490196078432, 0.9254901960784314),
 (0.5372549019607843, 0.803921568627451, 0.4745098039215686),
 (0.8627450980392157, 0.4117647058823529, 0.48627450980392156)]

In [3]:
deploy_type = "local"

config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])
gait_data_path = Path(config_dict[deploy_type]["gait_data_path"])

base_path

PosixPath('/home/rzlin/ys64ofuj/Data/StressGait/Data')

In [4]:
dataset = StressGaitDataset(base_path, coarse_condition=True, gait_data_path=gait_data_path, specify_bouts=True, specify_speed=True)

## computation

In [5]:
data = []
for trial in dataset:

    keypoints = trial.load_keypoint_trajectories()
    hs_events = compute_HS(keypoints)
    hs_events = [i for i in hs_events if i > 30]
    stride_times = np.diff(hs_events)
    data.extend([
        (*trial.index.values[0], i, st)
        for i, st in zip(range(len(stride_times)), stride_times)
    ])

## store as multilevel df

In [9]:
stride_df = pd.DataFrame(data, columns=[*dataset.index.columns, "stride_idx", "stride_time"])
stride_df.set_index([*dataset.index.columns, "stride_idx"], inplace=True)
filename = dataset.base_path.joinpath("kinematics/stride_times.csv")
stride_df.to_csv(filename)

In [36]:
mean_stride_time = stride_df.groupby(["participant", "condition"]).mean()
stride_time_variability = stride_df.groupby(["participant", "condition"]).std()
mean_stride_time

,,stride_time
participant,condition,
VP_01,omc,1.169091
VP_02,control,1.121491
VP_05,control,1.134398
VP_06,omc,1.135896
VP_07,control,1.128527
VP_08,control,1.230708
VP_11,control,1.095589
VP_12,control,1.130466
VP_13,omc,1.132073


In [31]:
mean = mean_stride_time.groupby(["condition"]).mean()
mean

,stride_time
condition,
control,1.158599
omc,1.188345


In [32]:
normality_results = mean_stride_time.groupby(['condition'], group_keys=False)["stride_time"].apply(lambda g: pg.normality(g))
normality_results

,W,pval,normal
control,0.929832,0.153271,True
omc,0.944561,0.291953,True


In [33]:
normality_results_var = stride_time_variability.groupby(['condition'], group_keys=False)["stride_time"].apply(lambda g: pg.normality(g))
normality_results_var

,W,pval,normal
control,0.957412,4.935340e-01,True
omc,0.551474,9.627790e-07,False


In [38]:
test_results = pg.pairwise_tests(data=mean_stride_time.reset_index(),
                                 dv='stride_time',
                                 between='condition',
                                 parametric=False)
test_results["p_corr"] = test_results["p-unc"]*2
test_results["p_corr"].clip(upper=1, inplace=True)
test_results

/tmp/ipykernel_76572/599718129.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_results["p_corr"].clip(upper=1, inplace=True)


,Contrast,A,B,Paired,Parametric,U-val,alternative,p-unc,hedges,p_corr
0,condition,control,omc,False,False,142.0,two-sided,0.119856,-0.423852,0.239711
